# Dynamic WSCC 9-bus System with Switch

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://git.rwth-aachen.de/acs/public/grid-data/cim-grid-data/-/raw/master/WSCC-09/WSCC-09_RX_Dyn/WSCC-09_RX'
filename = 'WSCC-09'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
import villas.dataprocessing.plottools as pt
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import dpsimpy
import numpy as np

In [ ]:
sim_name = 'SP_WSCC-9bus_dyn_switch'

## Simulation

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system = reader.loadCIM(60, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.TransientStability)

# Extend topology with switch
sw = dpsimpy.sp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
sw.set_parameters(1e12, 0.1*529)
sw.connect([dpsimpy.sp.SimNode.gnd, system.node('BUS7')])
sw.open()
system.add(sw)

# Use omegNom for torque conversion in SG models for validation with PSAT
system.component('GEN1').set_model_flags(False, False)
system.component('GEN2').set_model_flags(False, False)
system.component('GEN3').set_model_flags(False, False)

logger = dpsimpy.Logger(sim_name)

for i in range(1, 10):
    logger.log_attribute('v' + str(i), 'v', system.node('BUS' + str(i)))

for i in range(1, 4):
    logger.log_attribute('wr_' + str(i), 'w_r', system.component('GEN' + str(i)))
    logger.log_attribute('delta_r_' + str(i), 'delta_r', system.component('GEN' + str(i)))
    logger.log_attribute('P_elec_' + str(i), 'P_elec', system.component('GEN' + str(i)))
    logger.log_attribute('P_mech_' + str(i), 'P_mech', system.component('GEN' + str(i)))


sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(system)
sim.set_time_step(0.0001)
sim.set_final_time(2.0)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.do_init_from_nodes_and_terminals(True)

sw_event_1 = dpsimpy.event.SwitchEvent(0.2 - 0.0001, sw, True)
sim.add_event(sw_event_1)

sim.add_logger(logger)
sim.run()


## Read simulation results

In [ ]:
path = 'logs/SP_WSCC-9bus_dyn_switch/'
logName = 'SP_WSCC-9bus_dyn_switch'
logFilename = path + logName + '.csv'
print(logFilename)

ts_dpsim = rt.read_timeseries_dpsim(logFilename)
phasors = ts.phasors(ts_dpsim)

### Phasors at last time step in per unit

In [ ]:
nominal_voltages = {'v1': 16500, 'v2': 18000, 'v3': 13800, 
                         'v4': 230000, 'v5': 230000, 'v6': 230000, 
                         'v7': 230000, 'v8': 230000, 'v9': 230000} 
for node, nom_voltage in nominal_voltages.items():
    print(node + ': ' + str(phasors[node]['abs'].values[0] / nom_voltage) + '<' + str(phasors[node]['phase'].values[0]))

### Plot node voltages

In [ ]:
plt.figure()
plt.plot(phasors['v1']['abs'].time, phasors['v1']['abs'].values, label='v1.abs')
plt.plot(phasors['v2']['abs'].time, phasors['v2']['abs'].values, label='v2.abs')
plt.plot(phasors['v3']['abs'].time, phasors['v3']['abs'].values, label='v3.abs')
plt.legend()
plt.ylim([0,20000])

plt.figure()
plt.plot(phasors['v4']['abs'].time, phasors['v4']['abs'].values, label='v4.abs')
plt.plot(phasors['v5']['abs'].time, phasors['v5']['abs'].values, label='v5.abs')
plt.plot(phasors['v6']['abs'].time, phasors['v6']['abs'].values, label='v6.abs')
plt.plot(phasors['v7']['abs'].time, phasors['v7']['abs'].values, label='v7.abs')
plt.plot(phasors['v8']['abs'].time, phasors['v8']['abs'].values, label='v8.abs')
plt.plot(phasors['v9']['abs'].time, phasors['v9']['abs'].values, label='v9.abs')
plt.legend()
plt.ylim([0,240000])

### Generator Speed

In [ ]:
plt.figure()
plt.plot(ts_dpsim['wr_1'].time, ts_dpsim['wr_1'].values, label='wr_1')
plt.plot(ts_dpsim['wr_2'].time, ts_dpsim['wr_2'].values, label='wr_2')
plt.plot(ts_dpsim['wr_3'].time, ts_dpsim['wr_3'].values, label='wr_3')
plt.xlabel('time (s)')
plt.ylabel('mechanical speed (rad/s)')
plt.legend()
plt.show()
#plt.savefig('wscc_9bus_fault_gen_speed.pdf')

## Validation with PSAT

In [ ]:
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/PSAT/WSCC-9bus/d_009_fault_dpsim_01.out'
local_file = 'reference-results/d_009_fault_dpsim_01.out'
urllib.request.urlretrieve(url, local_file) 

syngen_power_name_dpsim_list = ['P_elec_1', 'P_elec_2', 'P_elec_3']
syngen_power_name_psat_list = ['p_Syn_1', 'p_Syn_2', 'p_Syn_3']

syngen_omega_name_dpsim_list = ['wr_1', 'wr_2', 'wr_3']
syngen_omega_name_psat_list = ['omega_Syn_1', 'omega_Syn_2', 'omega_Syn_3']

syngen_delta_name_dpsim_list = ['delta_r_1', 'delta_r_2', 'delta_r_3']
syngen_delta_name_psat_list = ['delta_Syn_1', 'delta_Syn_2', 'delta_Syn_3']

bus_volt_name_dpsim_list = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9']
bus_volt_name_psat_list = ['V_Bus 1', 'V_Bus 2', 'V_Bus 3', 'V_Bus 4', 'V_Bus 5', 'V_Bus 6', 'V_Bus 7', 'V_Bus 8', 'V_Bus 9']
bus_angle_name_psat_list = ['theta_Bus 1', 'theta_Bus 2', 'theta_Bus 3', 'theta_Bus 4', 'theta_Bus 5', 'theta_Bus 6', 'theta_Bus 7', 'theta_Bus 8', 'theta_Bus 9']

timeseries_names_psat = syngen_power_name_psat_list+syngen_omega_name_psat_list+syngen_delta_name_psat_list+bus_volt_name_psat_list+bus_angle_name_psat_list

ts_psat = rt.read_timeseries_PSAT(local_file, timeseries_names_psat)

## Rotor speeds

In [ ]:
plt.figure(figsize=(12,9))
for syngen_omega_name_dpsim in syngen_omega_name_dpsim_list:
    ts_dpsim[syngen_omega_name_dpsim].values = ts_dpsim[syngen_omega_name_dpsim].values/(2*np.pi*60)
    plt.plot(ts_dpsim[syngen_omega_name_dpsim].time, ts_dpsim[syngen_omega_name_dpsim].values, label=syngen_omega_name_dpsim+' (dpsim)')
for syngen_omega_name_psat in syngen_omega_name_psat_list:
    plt.plot(ts_psat[syngen_omega_name_psat].time, ts_psat[syngen_omega_name_psat].values, label=syngen_omega_name_psat+' (psat)', linestyle='--')
plt.xlabel('time (s)')
plt.ylabel('mechanical speed (p.u)')
plt.legend()
plt.show()

## Assert rotor speeds

In [ ]:
assert(ts_dpsim[syngen_omega_name_dpsim_list[0]].rmse(ts_dpsim[syngen_omega_name_dpsim_list[0]], ts_psat[syngen_omega_name_psat_list[2]]) < 2e-4)
assert(ts_dpsim[syngen_omega_name_dpsim_list[1]].rmse(ts_dpsim[syngen_omega_name_dpsim_list[1]], ts_psat[syngen_omega_name_psat_list[0]]) < 2e-4)
assert(ts_dpsim[syngen_omega_name_dpsim_list[2]].rmse(ts_dpsim[syngen_omega_name_dpsim_list[2]], ts_psat[syngen_omega_name_psat_list[1]]) < 2e-4)

## Rotor angles

In [ ]:
plt.figure(figsize=(12,9))
for syngen_delta_name_dpsim in syngen_delta_name_dpsim_list:
    plt.plot(ts_dpsim[syngen_delta_name_dpsim].time, ts_dpsim[syngen_delta_name_dpsim].values, label=syngen_delta_name_dpsim+' (dpsim)')
for syngen_delta_name_psat in syngen_delta_name_psat_list:
    plt.plot(ts_psat[syngen_delta_name_psat].time, ts_psat[syngen_delta_name_psat].values, label=syngen_delta_name_psat+' (psat)', linestyle='--')
plt.xlabel('time (s)')
plt.ylabel('rotor angle (deg)')
plt.legend()
plt.show()

## Bus voltages

In [ ]:
plt.figure(figsize=(12,9))
for bus_volt_name_dpsim in bus_volt_name_dpsim_list:
    if bus_volt_name_dpsim == 'v1':
        plt.plot(ts_dpsim[bus_volt_name_dpsim].time, ts_dpsim[bus_volt_name_dpsim].abs().values/16.5e3, label=bus_volt_name_dpsim + ' (dpsim)', color='C'+str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)))
    elif bus_volt_name_dpsim == 'v2':
        plt.plot(ts_dpsim[bus_volt_name_dpsim].time, ts_dpsim[bus_volt_name_dpsim].abs().values/18e3, label=bus_volt_name_dpsim + ' (dpsim)', color='C'+str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)))
    elif bus_volt_name_dpsim == 'v3':
        plt.plot(ts_dpsim[bus_volt_name_dpsim].time, ts_dpsim[bus_volt_name_dpsim].abs().values/13.8e3, label=bus_volt_name_dpsim + ' (dpsim)', color='C'+str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)))
    else:
        plt.plot(ts_dpsim[bus_volt_name_dpsim].time, ts_dpsim[bus_volt_name_dpsim].abs().values/230e3, label=bus_volt_name_dpsim + ' (dpsim)', color='C'+str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)))
for bus_volt_name_psat in bus_volt_name_psat_list:
        plt.plot(ts_psat[bus_volt_name_psat].time, ts_psat[bus_volt_name_psat].values, label=bus_volt_name_psat + ' (psat)', linestyle='--', color='C'+str(bus_volt_name_psat_list.index(bus_volt_name_psat)))
plt.xlabel('time (s)')
plt.ylabel('voltage (p.u.)')
plt.legend()
plt.show()

## Bus angles

In [ ]:
plt.figure(figsize=(12,9))
for bus_volt_name_dpsim in bus_volt_name_dpsim_list:
    plt.plot(ts_dpsim[bus_volt_name_dpsim].time, ts_dpsim[bus_volt_name_dpsim].phase().values/180*np.pi, label=bus_volt_name_dpsim + ' (dpsim)', color='C'+str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)))
for bus_angle_name_psat in bus_angle_name_psat_list:
    plt.plot(ts_psat[bus_angle_name_psat].time, ts_psat[bus_angle_name_psat].values, label=bus_angle_name_psat + ' (psat)', linestyle='--', color='C'+str(bus_angle_name_psat_list.index(bus_angle_name_psat)))
plt.xlabel('time (s)')
plt.ylabel('angle (rad)')
plt.legend()
plt.show()

## SG active power

In [ ]:
plt.figure(figsize=(12,9))
for syngen_power_name_dpsim in syngen_power_name_dpsim_list:
    plt.plot(ts_dpsim[syngen_power_name_dpsim].time, ts_dpsim[syngen_power_name_dpsim].values/100e6, label=syngen_power_name_dpsim+' (dpsim)')
for syngen_power_name_psat in syngen_power_name_psat_list:
    plt.plot(ts_psat[syngen_power_name_psat].time, ts_psat[syngen_power_name_psat].values, label=syngen_power_name_psat+' (psat)', linestyle='--')
plt.xlabel('time (s)')
plt.ylabel('power (W)')
plt.legend()
plt.show()